In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
cats = ['alt.atheism','sci.space']
newsgroups_data = fetch_20newsgroups(subset='all',shuffle=True,random_state=42,categories=cats)

X_train,X_test,y_train,y_test = train_test_split(newsgroups_data.data,newsgroups_data.target,test_size=0.2,random_state=42)

vectorizer = CountVectorizer(stop_words='english')
X_train = vectorizer.fit_transform(X_train).toarray()
X_test = vectorizer.transform(X_test).toarray()

X_train = torch.tensor(X_train,dtype=torch.float32)
y_train = torch.tensor(y_train,dtype=torch.long)
X_test = torch.tensor(X_test,dtype=torch.float32)
y_test = torch.tensor(y_test,dtype=torch.long)

In [3]:
class TextClassifier(nn.Module):
    def __init__(self,num_classes):
        super(TextClassifier,self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1],128)
        self.fc2 = nn.Linear(128,num_classes)
    
    def forward(self,x):
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return nn.functional.log_softmax(x,dim=1)

In [4]:
model = TextClassifier(num_classes=len(cats))
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.01)

num_epochs = 1
batch_size = 10
num_batches = len(X_train) // batch_size

for epoch in range(num_epochs):
    total_loss = 0.0
    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = (i+1) * batch_size
        inputs = X_train[start_idx:end_idx]
        targets = y_train[start_idx:end_idx]
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs,targets)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

test_output = model(X_test)
test_predictions = torch.argmax(test_output,dim=1)
test_accuracy = accuracy_score(y_test,test_predictions)

print(f"Epoch: {epoch+1} , Loss: {total_loss/num_batches} , Test Accuracy:{test_accuracy}")

Epoch: 1 , Loss: 0.15354424601696967 , Test Accuracy:0.9916201117318436


Discussion

In [5]:
X_train.shape[1]

25150